In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import json

from sklearn.metrics import roc_auc_score, average_precision_score

PROCESSED_PATH = "../data/processed/"

X_val  = pd.read_csv(PROCESSED_PATH + "X_val.csv")
y_val  = pd.read_csv(PROCESSED_PATH + "y_val.csv").squeeze("columns")

X_test = pd.read_csv(PROCESSED_PATH + "X_test.csv")
y_test = pd.read_csv(PROCESSED_PATH + "y_test.csv").squeeze("columns")

calibrator = joblib.load("../artifacts/models/calibrator_final.joblib")

pd_val  = calibrator.predict_proba(X_val)[:, 1]
pd_test = calibrator.predict_proba(X_test)[:, 1]

print("VAL ROC-AUC:", roc_auc_score(y_val, pd_val))
print("VAL PR-AUC:", average_precision_score(y_val, pd_val))
print("TEST ROC-AUC:", roc_auc_score(y_test, pd_test))
print("TEST PR-AUC:", average_precision_score(y_test, pd_test))


VAL ROC-AUC: 0.7754295185460706
VAL PR-AUC: 0.524960430411155
TEST ROC-AUC: 0.7846422819541979
TEST PR-AUC: 0.5504181514096691


In [2]:
# Costs (adjust later if you want different behavior)
C_DEFAULT_APPROVED = 100   # big loss
C_GOOD_REJECTED    = 10    # opportunity cost
C_REVIEW           = 2     # operational cost

def decision_from_pd(pd, t_appr, t_rej):
    if pd < t_appr:
        return "APPROVE"
    elif pd < t_rej:
        return "REVIEW"
    else:
        return "REJECT"

def total_cost(y_true, pd, t_appr, t_rej):
    y_true = np.asarray(y_true)
    pd = np.asarray(pd)

    approve = pd < t_appr
    review  = (pd >= t_appr) & (pd < t_rej)
    reject  = pd >= t_rej

    cost_default_approved = C_DEFAULT_APPROVED * np.sum(approve & (y_true == 1))
    cost_good_rejected    = C_GOOD_REJECTED    * np.sum(reject  & (y_true == 0))
    cost_review           = C_REVIEW           * np.sum(review)

    return float(cost_default_approved + cost_good_rejected + cost_review)


In [3]:
def policy_outcomes(y_true, pd, t_appr, t_rej):
    y_true = np.asarray(y_true)
    pd = np.asarray(pd)

    approve = pd < t_appr
    review  = (pd >= t_appr) & (pd < t_rej)
    reject  = pd >= t_rej

    out = {
        "approve_rate": float(approve.mean()),
        "review_rate": float(review.mean()),
        "reject_rate": float(reject.mean()),
        "default_rate_overall": float(y_true.mean()),
        "default_rate_approved": float(y_true[approve].mean()) if approve.sum() else None,
        "default_rate_review": float(y_true[review].mean()) if review.sum() else None,
        "default_rate_reject": float(y_true[reject].mean()) if reject.sum() else None,
        "n_approved": int(approve.sum()),
        "n_review": int(review.sum()),
        "n_reject": int(reject.sum()),
        "total_cost": float(total_cost(y_true, pd, t_appr, t_rej)),
        "t_approve": float(t_appr),
        "t_reject": float(t_rej)
    }
    return out


In [4]:
t_appr_grid = np.linspace(0.05, 0.35, 31)
t_rej_grid  = np.linspace(0.20, 0.70, 51)

MAX_REVIEW_RATE = None   # set to something like 0.35 if you want capacity constraints

best = None
best_cost = float("inf")

for t_appr in t_appr_grid:
    for t_rej in t_rej_grid:
        if t_rej <= t_appr:
            continue

        out = policy_outcomes(y_val, pd_val, t_appr, t_rej)

        if MAX_REVIEW_RATE is not None and out["review_rate"] > MAX_REVIEW_RATE:
            continue

        if out["total_cost"] < best_cost:
            best_cost = out["total_cost"]
            best = (t_appr, t_rej)

best_t_approve, best_t_reject = best
print("Best thresholds (from VAL):", best_t_approve, best_t_reject)
print("Best VAL cost:", best_cost)


Best thresholds (from VAL): 0.05 0.6299999999999999
Best VAL cost: 12256.0


In [5]:
val_policy  = policy_outcomes(y_val, pd_val, best_t_approve, best_t_reject)
test_policy = policy_outcomes(y_test, pd_test, best_t_approve, best_t_reject)

print("\nVAL policy:")
val_policy

print("\nTEST policy (final):")
test_policy



VAL policy:

TEST policy (final):


{'approve_rate': 0.043666666666666666,
 'review_rate': 0.9121666666666667,
 'reject_rate': 0.04416666666666667,
 'default_rate_overall': 0.22116666666666668,
 'default_rate_approved': 0.04198473282442748,
 'default_rate_review': 0.20317924355929107,
 'default_rate_reject': 0.769811320754717,
 'n_approved': 262,
 'n_review': 5473,
 'n_reject': 265,
 'total_cost': 12656.0,
 't_approve': 0.05,
 't_reject': 0.6299999999999999}

In [6]:
os.makedirs("../artifacts/reports", exist_ok=True)

thresholds = {
    "model": "calibrator_final.joblib",
    "t_approve": float(best_t_approve),
    "t_reject": float(best_t_reject),
    "costs": {
        "C_DEFAULT_APPROVED": float(C_DEFAULT_APPROVED),
        "C_GOOD_REJECTED": float(C_GOOD_REJECTED),
        "C_REVIEW": float(C_REVIEW)
    }
}

with open("../artifacts/reports/thresholds_final.json", "w") as f:
    json.dump(thresholds, f, indent=2)

with open("../artifacts/reports/policy_summary_test.json", "w") as f:
    json.dump(test_policy, f, indent=2)

print("Saved:")
print("- ../artifacts/reports/thresholds_final.json")
print("- ../artifacts/reports/policy_summary_test.json")


Saved:
- ../artifacts/reports/thresholds_final.json
- ../artifacts/reports/policy_summary_test.json
